## _Instalación de los datasets_

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

Path to dataset files: /Users/victoros/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/2


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("terencicp/e-commerce-dataset-by-olist-as-an-sqlite-database")

print("Path to dataset files:", path)

100%|██████████| 48.7M/48.7M [00:01<00:00, 32.9MB/s]

Extracting files...


Path to dataset files: /Users/victoros/.cache/kagglehub/datasets/terencicp/e-commerce-dataset-by-olist-as-an-sqlite-database/versions/1


# 1) Carga de los datos y GMV mensual

In [5]:
import pandas as pd

orders = pd.read_csv("data/olist_orders_dataset.csv", parse_dates=["order_purchase_timestamp"])
order_items = pd.read_csv("data/olist_order_items_dataset.csv")

print(orders.head())
print(order_items.head())

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00           2018-08

Unimos ambos dataframes y agregamos las columnas necesarias para obtener el GMV mensual:

In [8]:
df = orders.merge(order_items, on="order_id", how="inner")

df["gmv_item"] = df["price"] + df["freight_value"]

df["year"] = df["order_purchase_timestamp"].dt.year
df["month"] = df["order_purchase_timestamp"].dt.month

In [10]:
gmv_mensual =  (df.groupby(["year", "month"], as_index=False)["gmv_item"].sum().rename(columns={"gmv_item" : "gmv_mensual"}))

print(gmv_mensual.head(10))

   year  month  gmv_mensual
0  2016      9       354.75
1  2016     10     56808.84
2  2016     12        19.62
3  2017      1    137188.49
4  2017      2    286280.62
5  2017      3    432048.59
6  2017      4    412422.24
7  2017      5    586190.95
8  2017      6    502963.04
9  2017      7    584971.62


# 2) Función limpiar_fechas()

Elimina registros donde la fecha de entrega es anterior a la fecha de compra.

In [14]:
def limpiar_fechas(df):
    df = df.copy()
    # Aseguramos el datetime
    df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"])
    df["order_delivered_customer_date"] = pd.to_datetime(df["order_purchase_timestamp"])

    mask_valid = (
            df["order_delivered_customer_date"].isna() | (df["order_delivered_customer_date"] >= df["order_purchase_timestamp"])
    )

    df_limpio = df[mask_valid].reset_index(drop=True)
    return df_limpio

df_clean = limpiar_fechas(df)

print(df.count().head())
print(df_clean.count().head())

order_id                    112650
customer_id                 112650
order_status                112650
order_purchase_timestamp    112650
order_approved_at           112635
dtype: int64
order_id                    112650
customer_id                 112650
order_status                112650
order_purchase_timestamp    112650
order_approved_at           112635
dtype: int64


# 10) Proporción de reseñas positivas

In [15]:
reviews = pd.read_csv("data/olist_order_reviews_dataset.csv", parse_dates=["review_creation_date"])

# Columna de mes (año-mes)
reviews["year_month"] = reviews["review_creation_date"].dt.to_period("M")

# Reseña positiva: review_score >= 4
reviews["positiva"] = reviews["review_score"] >= 4

# Proporción de reseñas positivas por mes
prop_positivas_mes = (
    reviews.groupby("year_month")["positiva"]
           .mean()
           .reset_index()
           .rename(columns={"positiva": "proporcion_positivas"})
)

print(prop_positivas_mes.head())


  year_month  proporcion_positivas
0    2016-10              0.765363
1    2016-11              0.544554
2    2016-12              0.333333
3    2017-01              0.820084
4    2017-02              0.819533


# 11) Distribución de payments_type

In [21]:
payments = pd.read_csv("data/olist_order_payments_dataset.csv")

# Distribución de tipos de pago (frecuencia relativa)
dist_payment_type = (
    payments["payment_type"]
    .value_counts(normalize=True)
    .rename("proporcion")
    .reset_index()
    .rename(columns={"index": "payment_type"})
)

print("Distribución de payment_type:")
print(dist_payment_type)

# Ticket promedio por metodo de pago (promedio del payment_value)
ticket_promedio = (
    payments.groupby("payment_type", as_index=False)["payment_value"]
            .mean()
            .rename(columns={"payment_value": "ticket_promedio"})
)
print("======================================")
print("Ticket promedio por método de pago:")
print(ticket_promedio)

Distribución de payment_type:
  payment_type  proporcion
0  credit_card    0.739224
1       boleto    0.190440
2      voucher    0.055590
3   debit_card    0.014718
4  not_defined    0.000029
Ticket promedio por método de pago:
  payment_type  ticket_promedio
0       boleto       145.034435
1  credit_card       163.319021
2   debit_card       142.570170
3  not_defined         0.000000
4      voucher        65.703354


# 12) Diferencia de inner join vs. left join

- El _inner join_ solo conserva las filas donde coinciden ambos Dataframes, los demas se descartan.
- El _left join_ conserva todas las filas del Dataframe de la izquierda y inserta _NaN_ donde no hay coincidencias en el DataFrame de la derecha.

In [29]:
df_orders = pd.DataFrame({
    "order_id": [1, 2, 3],
    "customer": ["Ana", "Luis", "Pedro"]
})

df_payments = pd.DataFrame({
    "order_id": [1, 3, 4],
    "payment": ["card", "boleto", "card"]
})

# Inner join
df_inner = pd.merge(df_orders, df_payments, on="order_id", how="inner")
print(df_inner.head())

# Left join
df_left_join = pd.merge(df_orders, df_payments, on="order_id", how="left")
print(df_left_join.head())

   order_id customer payment
0         1      Ana    card
1         3    Pedro  boleto
   order_id customer payment
0         1      Ana    card
1         2     Luis     NaN
2         3    Pedro  boleto
